# Lezione 2: Checkpointing e Persistenza con LangGraph

## Obiettivi di Apprendimento
In questa lezione imparerai:
1. **Cos'è il checkpointing** e perché è importante
2. **Come configurare la persistenza** dello stato in LangGraph
3. **Crash recovery**: come riprendere un workflow interrotto
4. **Time-travel debugging**: navigare la storia degli stati
5. **Quando usare LangGraph** invece di semplici agenti ReAct

## Scenario: Sistema di Prenotazione Resiliente
Creeremo un sistema di prenotazione viaggio multi-step che può:
- Salvare automaticamente il progresso ad ogni passo
- Riprendere da dove si era interrotto in caso di crash
- Permettere di tornare indietro e modificare scelte precedenti

## 1. Setup Iniziale

In [ ]:
# Installa le dipendenze necessarie
# %pip install -qU langgraph langchain langchain-cerebras tavily-python

In [ ]:
import os
from datetime import datetime
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_cerebras import ChatCerebras
from langchain_tavily import TavilySearch
from langchain.agents import create_agent

# Configura le API keys
from dotenv import load_dotenv
load_dotenv()

## 2. Definizione dello State

Lo state tiene traccia di tutto il processo di prenotazione multi-step:

In [ ]:
class TravelBookingState(TypedDict):
    """State per il workflow di prenotazione viaggio.
    
    Ogni campo rappresenta uno step del processo:
    - messages: conversazione con l'utente
    - destination: città di destinazione scelta
    - hotel_info: risultati ricerca hotel
    - activities_info: attività consigliate
    - booking_confirmed: flag di conferma finale
    - step_completed: contatore dei passi completati
    """
    messages: Annotated[Sequence[BaseMessage], add_messages]
    destination: str
    hotel_info: str
    activities_info: str
    booking_confirmed: bool
    step_completed: int

## 3. Configurazione degli Agenti

Creiamo agenti specializzati per ogni fase del booking:

In [ ]:
# Modello LLM
llm = ChatCerebras(model="llama-3.3-70b")

# Tool per ricerca web
search_tool = TavilySearch(max_results=2)

# Agente per ricerca hotel
hotel_agent = create_agent(
    model=llm,
    tools=[search_tool],
    system_prompt="""Sei un esperto di hotel. 
    Cerca i migliori hotel nella destinazione richiesta.
    Fornisci 2-3 opzioni con prezzo indicativo e caratteristiche.
    Rispondi in modo conciso e strutturato."""
)

# Agente per attività turistiche
activities_agent = create_agent(
    model=llm,
    tools=[search_tool],
    system_prompt="""Sei una guida turistica esperta.
    Suggerisci le migliori attività e luoghi da visitare.
    Considera la stagione e gli interessi del viaggiatore.
    Fornisci 3-5 suggerimenti concreti."""
)

## 4. Definizione dei Nodi del Workflow

Ogni nodo rappresenta uno step salvabile del processo:

In [ ]:
def collect_destination(state: TravelBookingState) -> TravelBookingState:
    """Step 1: Raccolta destinazione dall'utente."""
    print("📍 STEP 1: Raccolta destinazione...")
    
    # Estrai la destinazione dal messaggio utente
    user_msg = state["messages"][-1].content
    
    # Usa LLM per estrarre la destinazione
    extraction_prompt = f"""Estrai la città di destinazione da questo messaggio: '{user_msg}'.
    Rispondi SOLO con il nome della città, nient'altro."""
    
    destination = llm.invoke([HumanMessage(content=extraction_prompt)]).content.strip()
    
    return {
        "destination": destination,
        "step_completed": 1,
        "messages": [SystemMessage(content=f"✅ Destinazione registrata: {destination}")]
    }


def search_hotels(state: TravelBookingState) -> TravelBookingState:
    """Step 2: Ricerca hotel nella destinazione."""
    print(f"🏨 STEP 2: Ricerca hotel a {state['destination']}...")
    
    query = f"best hotels in {state['destination']} with prices"
    result = hotel_agent.invoke({
        "messages": [HumanMessage(content=query)]
    })
    
    hotel_info = result["messages"][-1].content
    
    return {
        "hotel_info": hotel_info,
        "step_completed": 2,
        "messages": [SystemMessage(content=f"✅ Hotel trovati:\n{hotel_info[:200]}...")]
    }


def search_activities(state: TravelBookingState) -> TravelBookingState:
    """Step 3: Ricerca attività turistiche."""
    print(f"🎭 STEP 3: Ricerca attività a {state['destination']}...")
    
    query = f"best things to do and activities in {state['destination']}"
    result = activities_agent.invoke({
        "messages": [HumanMessage(content=query)]
    })
    
    activities_info = result["messages"][-1].content
    
    return {
        "activities_info": activities_info,
        "step_completed": 3,
        "messages": [SystemMessage(content=f"✅ Attività trovate:\n{activities_info[:200]}...")]
    }


def confirm_booking(state: TravelBookingState) -> TravelBookingState:
    """Step 4: Conferma finale della prenotazione."""
    print("✅ STEP 4: Conferma prenotazione...")
    
    summary = f"""🎉 PRENOTAZIONE COMPLETATA!
    
📍 Destinazione: {state['destination']}
🏨 Hotel: {state['hotel_info'][:150]}...
🎭 Attività: {state['activities_info'][:150]}...

Grazie per aver usato il nostro servizio!
    """
    
    return {
        "booking_confirmed": True,
        "step_completed": 4,
        "messages": [SystemMessage(content=summary)]
    }

## 5. Costruzione del Grafo con Checkpointing

**QUESTO È IL PUNTO CHIAVE**: Usiamo `MemorySaver` per salvare automaticamente lo stato ad ogni passo.

In [ ]:
# Crea il grafo
workflow = StateGraph(TravelBookingState)

# Aggiungi i nodi
workflow.add_node("collect_destination", collect_destination)
workflow.add_node("search_hotels", search_hotels)
workflow.add_node("search_activities", search_activities)
workflow.add_node("confirm_booking", confirm_booking)

# Definisci il flusso sequenziale
workflow.add_edge(START, "collect_destination")
workflow.add_edge("collect_destination", "search_hotels")
workflow.add_edge("search_hotels", "search_activities")
workflow.add_edge("search_activities", "confirm_booking")
workflow.add_edge("confirm_booking", END)

# 🔑 CHIAVE: Configura il checkpointer per la persistenza
memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory,
)

print("✅ Grafo compilato con checkpointing e human-in-the-loop abilitati!")


## 6. Visualizza il Grafo

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Nota: Per visualizzare il grafo installa: pip install grandalf")
    print(f"Errore: {e}")

## 7. Esecuzione Normale (Senza Interruzioni)

Prima vediamo come funziona in condizioni normali:

In [ ]:
# Configurazione con thread_id per identificare la sessione
config = {"configurable": {"thread_id": "viaggio_001"}}

# Input iniziale
initial_input = {
    "messages": [HumanMessage(content="Voglio prenotare un viaggio a Parigi")],
    "destination": "",
    "hotel_info": "",
    "activities_info": "",
    "booking_confirmed": False,
    "step_completed": 0
}

print("🚀 Avvio workflow di prenotazione...\n")
print("=" * 60)

# Esegui il workflow
for event in app.stream(initial_input, config, stream_mode="values"):
    if "messages" in event and event["messages"]:
        last_msg = event["messages"][-1]
        print(f"\n{last_msg.content}")
        print("-" * 60)

## 8. Ispeziona i Checkpoint Salvati

Ogni step è stato salvato automaticamente. Vediamoli:

In [ ]:
# Recupera tutti i checkpoint per questo thread
checkpoints = list(app.get_state_history(config))

print(f"📊 Trovati {len(checkpoints)} checkpoint salvati:\n")
print("=" * 80)

for i, checkpoint in enumerate(checkpoints):
    state = checkpoint.values
    print(f"\n🔖 CHECKPOINT {i + 1}:")
    print(f"   - Step completato: {state.get('step_completed', 0)}/4")
    print(f"   - Destinazione: {state.get('destination', 'N/A')}")
    print(f"   - Hotel trovati: {'✅' if state.get('hotel_info') else '❌'}")
    print(f"   - Attività trovate: {'✅' if state.get('activities_info') else '❌'}")
    print(f"   - Booking confermato: {'✅' if state.get('booking_confirmed') else '❌'}")
    print(f"   - Config: {checkpoint.config}")
    print("-" * 80)

## 9. Simulazione di Crash e Recovery

**Scenario**: Il sistema crasha dopo aver trovato gli hotel ma prima di cercare le attività.
Possiamo riprendere esattamente da dove ci eravamo fermati!

In [ ]:
# Simula un nuovo workflow che crasha dopo step 2
config_crash = {"configurable": {"thread_id": "viaggio_002_crash"}}

initial_input_2 = {
    "messages": [HumanMessage(content="Voglio andare a Tokyo")],
    "destination": "",
    "hotel_info": "",
    "activities_info": "",
    "booking_confirmed": False,
    "step_completed": 0
}

print("🚀 Avvio workflow che crasherà...\n")
print("=" * 60)

# Esegui solo fino allo step 2
step_count = 0
for event in app.stream(initial_input_2, config_crash, stream_mode="values"):
    step_count += 1
    if "messages" in event and event["messages"]:
        last_msg = event["messages"][-1]
        print(f"\n{last_msg.content}")
    
    # Simula crash dopo step 2 (hotel trovati)
    if step_count >= 2:
        print("\n💥 CRASH! Sistema terminato inaspettatamente...")
        break

print("\n" + "=" * 60)

In [ ]:
# RECOVERY: Ripristina lo stato e riprendi da dove ci eravamo fermati
print("\n🔄 RECOVERY: Ripristino dello stato salvato...\n")
print("=" * 60)

# Recupera lo stato corrente
current_state = app.get_state(config_crash)
print(f"📊 Stato recuperato:")
print(f"   - Step completato: {current_state.values.get('step_completed', 0)}/4")
print(f"   - Destinazione: {current_state.values.get('destination')}")
print(f"   - Hotel info presente: {'✅' if current_state.values.get('hotel_info') else '❌'}")
print(f"   - Activities info presente: {'✅' if current_state.values.get('activities_info') else '❌'}")

print("\n🚀 Ripresa esecuzione dal checkpoint...\n")
print("=" * 60)

# Riprendi esecuzione (senza input, usa lo stato salvato)
for event in app.stream(None, config_crash, stream_mode="values"):
    if "messages" in event and event["messages"]:
        last_msg = event["messages"][-1]
        print(f"\n{last_msg.content}")
        print("-" * 60)

print("\n✅ Recovery completato con successo!")

## 10. Time-Travel Debugging

Possiamo "tornare indietro nel tempo" a qualsiasi checkpoint precedente:

In [ ]:
# Recupera tutti i checkpoint del primo workflow
#config_original = {"configurable": {"thread_id": "viaggio_001"}}
history = list(app.get_state_history(config))

print(f"📜 Storia disponibile: {len(history)} checkpoint\n")

# Torna indietro al checkpoint dopo il primo step (destinazione raccolta)
if len(history) >= 3:
    checkpoint_to_restore = history[-2]  # Step 1 completato
    
    print(f"⏮️  Torno al checkpoint dopo step 1...\n")
    print("Stato a quel punto:")
    print(f"   - Destinazione: {checkpoint_to_restore.values.get('destination')}")
    print(f"   - Hotel: {'Trovati' if checkpoint_to_restore.values.get('hotel_info') else 'Non ancora'}")
    print(f"   - Step: {checkpoint_to_restore.values.get('step_completed')}")
    
    # Puoi continuare da quel punto con modifiche
    print("\n💡 Da qui potresti:")
    print("   - Cambiare la destinazione")
    print("   - Modificare i criteri di ricerca")
    print("   - Riprovare con parametri diversi")

In [ ]:
# Posso riprendere l'esecuzione da quel checkpoint

for event in app.stream(None, checkpoint_to_restore.config, stream_mode="values"):
    if "messages" in event and event["messages"]:
        last_msg = event["messages"][-1]
        print(f"\n{last_msg.content}")
        print("-" * 60)

In [ ]:
checkpoint_to_restore.values['messages']

In [ ]:
# Oppure posso fare un fork

# Modifica solo ciò che ti serve
new_config = app.update_state(
    checkpoint_to_restore.config,
    {
        "messages": HumanMessage(content="In realtà vorrei andare a Londra",
                                 id= checkpoint_to_restore.values['messages'][0].id),
    }
)

# Riprendi da dove hai modificato
for event in app.stream(None, new_config, stream_mode="values"):
    if "messages" in event and event["messages"]:
        last_msg = event["messages"][-1]
        print(f"\n{last_msg.content}")
        print("-" * 60)

## 11. Human-in-the-Loop: Approvazione Manuale

**Scenario**: Prima di confermare la prenotazione, mettiamo in pausa il workflow per far approvare (e modificare) i dati all'utente.

Grazie a `interrupt_before=["confirm_booking"]`, il grafo si ferma automaticamente e aspetta l'intervento umano.

In [ ]:
# Crea il grafo
workflow = StateGraph(TravelBookingState)

# Aggiungi i nodi
workflow.add_node("collect_destination", collect_destination)
workflow.add_node("search_hotels", search_hotels)
workflow.add_node("search_activities", search_activities)
workflow.add_node("confirm_booking", confirm_booking)

# Definisci il flusso sequenziale
workflow.add_edge(START, "collect_destination")
workflow.add_edge("collect_destination", "search_hotels")
workflow.add_edge("search_hotels", "search_activities")
workflow.add_edge("search_activities", "confirm_booking")
workflow.add_edge("confirm_booking", END)

# 🔑 CHIAVE: Configura il checkpointer per la persistenza
memory = MemorySaver()
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["confirm_booking"]  # ⏸️ Pausa prima della conferma per approvazione umana
)

print("✅ Grafo compilato con checkpointing e human-in-the-loop abilitati!")


In [ ]:
print("🎯 SCENARIO: Richiedi approvazione umana prima della conferma\n")
print("=" * 60)

# Nuovo workflow con interrupt
config_hitl = {"configurable": {"thread_id": "viaggio_003_hitl"}}

initial_input_3 = {
    "messages": [HumanMessage(content="Voglio andare a Barcellona")],
    "destination": "",
    "hotel_info": "",
    "activities_info": "",
    "booking_confirmed": False,
    "step_completed": 0
}

print("🚀 Avvio workflow con human-in-the-loop...\n")

# Esegui fino all'interrupt
for event in app.stream(initial_input_3, config_hitl, stream_mode="values"):
    if "messages" in event and event["messages"]:
        print(f"\n{event['messages'][-1].content}")
        print("-" * 60)

print("\n⏸️  WORKFLOW IN PAUSA - In attesa di approvazione umana")
print("=" * 60)

In [ ]:
# Ispeziona lo state prima della conferma
current = app.get_state(config_hitl)

print("\n📋 Riepilogo da approvare:")
print(f"  📍 Destinazione: {current.values['destination']}")
print(f"  🏨 Hotel: {current.values['hotel_info'][:200]}...")
print(f"  🎭 Attività: {current.values['activities_info'][:200]}...")
print(f"\n🔮 Next step: {current.next}")  # Mostra quale nodo eseguirà dopo
print(f"\n💡 Lo stato è salvato. Puoi:")
print("   1. Ispezionare i dati")
print("   2. Modificarli con update_state()")
print("   3. Continuare con stream(None, config)")
print("   4. Oppure annullare tutto")

In [ ]:
# Simula decisione umana: modifica i dati
print("\n👤 L'utente revisiona i dati...\n")
print("✏️  L'utente aggiunge una nota preferenza sull'hotel\n")

app.update_state(
    config_hitl,
    {
        "hotel_info": current.values['hotel_info'] + "\n\n✏️ NOTA UTENTE: Preferisco hotel vicino al centro storico con colazione inclusa",
        "messages": [SystemMessage(content="✅ Dati modificati e approvati dall'utente")]
    }
)

print("✅ Modifiche applicate allo state!")
print("=" * 60)

In [ ]:
# Continua dopo approvazione
print("\n🚀 Approvazione ricevuta - Ripresa esecuzione...\n")
print("=" * 60)

for event in app.stream(None, config_hitl, stream_mode="values"):
    if "messages" in event and event["messages"]:
        print(f"\n{event['messages'][-1].content}")
        print("-" * 60)

print("\n✅ Workflow completato con intervento umano!")
print("\n💡 Nota: Le modifiche dell'utente sono state salvate nel checkpoint")

## 12. Opzioni Avanzate di Interrupt

### Interrupt Before vs After:

```python
# Pausa PRIMA che il nodo esegua (per approvazione preventiva)
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["confirm_booking"]
)

# Pausa DOPO che il nodo esegue (per review del risultato)
app = workflow.compile(
    checkpointer=memory,
    interrupt_after=["search_hotels"]
)
```

### Interrupt Multipli:

```python
# Pausa in più punti del workflow
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["search_hotels", "search_activities", "confirm_booking"]
)
```

### Annullamento del Workflow:

```python
# Invece di continuare, l'utente può annullare
if user_wants_to_cancel:
    app.update_state(
        config,
        {
            "booking_confirmed": False,
            "messages": [SystemMessage(content="❌ Prenotazione annullata dall'utente")]
        }
    )
    # Non chiamare stream(None, config) → il workflow rimane in pausa
```

### 🎯 Quando Usare Human-in-the-Loop:
1. **Approvazioni critiche** (pagamenti, modifiche importanti)
2. **Validazione dati** prima di azioni irreversibili
3. **Scelte ambigue** dove serve giudizio umano
4. **Compliance e audit** per tracciare decisioni umane

## 13. Confronto: LangGraph vs Create Agent

### ❌ Senza LangGraph (solo create_agent):
```python
# Se crasha, devi ricominciare da zero
result = agent.invoke({"messages": [HumanMessage("prenota viaggio")]})
# 💥 Crash → tutto perduto, riparti da zero
```

### ✅ Con LangGraph + Checkpointing:
```python
# Ogni step è salvato automaticamente
app.stream(input, config)  # thread_id = "viaggio_001"
# 💥 Crash → riprendi esattamente dal punto dove eri
app.stream(None, config)  # continua dal checkpoint
```

### 🎯 Quando Usare LangGraph:
1. **Workflow lunghi** che richiedono tempo (minuti/ore)
2. **Processi critici** dove perdere progresso è costoso
3. **Debugging complesso** dove serve ispezionare ogni step
4. **Sistemi production** che devono essere resilient ai crash
5. **User experience** dove l'utente non deve ripetere tutto

## 14. Persistenza con SQLite (Bonus)

Per produzione, usa `SqliteSaver` invece di `MemorySaver`:

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

# Salva i checkpoint in un database SQLite
with SqliteSaver.from_conn_string(":memory:") as sqlite_saver:
    app_persistent = workflow.compile(checkpointer=sqlite_saver)
    
    print("✅ App compilata con persistenza SQLite")
    print("💾 I checkpoint sopravvivono ai restart del processo!")
    print("")
    print("Per usare un file invece che memoria:")
    print('SqliteSaver.from_conn_string("checkpoints.db")')

## 15. Riepilogo e Best Practices

### 📚 Cosa Abbiamo Imparato:
1. **Checkpointing automatico** con `MemorySaver` o `SqliteSaver`
2. **Crash recovery** riprendendo dal thread_id
3. **Time-travel debugging** navigando la storia degli stati
4. **Human-in-the-loop** con `interrupt_before` e `interrupt_after`
5. **Update state** per modifiche manuali durante l'esecuzione
6. **Differenza chiave** tra agenti semplici e LangGraph

### 🎯 Best Practices:
- Usa `thread_id` significativi (es: user_id + timestamp)
- Aggiungi `step_completed` per tracking granulare
- Implementa retry logic nei nodi critici
- Testa il recovery in condizioni di crash simulato
- Usa SQLite per produzione, MemorySaver per dev
- Configura `interrupt_before` per approvazioni critiche

### 💡 Quando NON Serve LangGraph:
- Task singoli e veloci (< 10 secondi)
- Nessun bisogno di persistenza
- Flusso lineare senza condizioni
- → In questi casi, `create_agent` è più semplice!

### 🚀 Esercizi per Te:
1. Aggiungi un nodo che può fallire con retry automatico
2. Implementa la modifica della destinazione tornando indietro
3. Crea un dashboard che mostra tutti i workflow attivi
4. Aggiungi timeout per step che impiegano troppo tempo
5. Prova a implementare l'annullamento del workflow dall'interrupt